In [81]:
from everything import *

In [82]:
agg_num=100
aggdf = pd.read_csv(f"aggregated_{agg_num}_std.csv", index_col =False)
data = aggdf[["acoustic_data", "time_to_failure"]].to_numpy()


In [88]:
from sklearn.model_selection import train_test_split
#maybe normalize about zero?
np.random.seed(43)
ratio = 0.2
num_channels = 1
n = len(data)
sample_size = int(150_000/agg_num)
split_data = np.array(np.split(data[:(n//sample_size)*sample_size], n//sample_size))
# labels are first time to appear 
split_inputs = split_data[:,:,0].reshape(-1, sample_size,1)
split_labels = split_data[:,0,1].reshape(-1,1)
p = np.random.permutation(len(split_inputs))
split_inputs, split_labels = split_inputs[p], split_labels[p]
train_num = int(len(split_inputs)*ratio)
train_X, test_X = split_inputs[train_num:], split_inputs[:train_num]
train_y, test_y = split_labels[train_num:], split_labels[:train_num]
train_X.shape, train_y.shape

((3356, 1500, 1), (3356, 1))

In [84]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Reshape, MaxPooling1D, GlobalAveragePooling1D, Dropout
model = Sequential()
#model_m.add(Reshape((sample_size, num_channels), input_shape=(3356)))
model.add(Conv1D(100, 10, activation='relu', input_shape=(sample_size, num_channels)))
model.add(Conv1D(100, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 10, activation='relu'))
model.add(Conv1D(160, 10, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))
model.summary(), model.input_shape, model.output_shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_37 (Conv1D)           (None, 1491, 100)         1100      
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 1482, 100)         100100    
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 494, 100)          0         
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 485, 160)          160160    
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 476, 160)          256160    
_________________________________________________________________
global_average_pooling1d_10  (None, 160)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 160)               0         
__________

(None, (None, 1500, 1), (None, 1))

In [85]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
model.fit(train_X, train_y, epochs=20)


Epoch 1/20
3356/3356 [==============================] - 4s 1ms/step - loss: 3.1947 - mean_absolute_error: 3.1947
Epoch 2/20
3356/3356 [==============================] - 3s 874us/step - loss: 2.3711 - mean_absolute_error: 2.3711
Epoch 3/20
3356/3356 [==============================] - 3s 868us/step - loss: 2.4008 - mean_absolute_error: 2.4008
Epoch 4/20
3356/3356 [==============================] - 3s 873us/step - loss: 2.3566 - mean_absolute_error: 2.3566
Epoch 5/20
3356/3356 [==============================] - 3s 873us/step - loss: 2.3959 - mean_absolute_error: 2.3959
Epoch 6/20
3356/3356 [==============================] - 3s 864us/step - loss: 2.3425 - mean_absolute_error: 2.3425
Epoch 7/20
3356/3356 [==============================] - 3s 865us/step - loss: 2.3251 - mean_absolute_error: 2.3251
Epoch 8/20
3356/3356 [==============================] - 3s 866us/step - loss: 2.3038 - mean_absolute_error: 2.3038
Epoch 9/20
3356/3356 [==============================] - 3s 867us/step - loss: 2.29

In [86]:
model.evaluate(test_X, test_y)

838/838 [==============================] - 0s 536us/step


[2.153720864248162, 2.153720864248162]

In [87]:
model.save(f"agg_std_{agg_num}_model_std_2.15")

In [72]:
model.predict(test_X[:10])

array([[0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422],
       [0.00043422]], dtype=float32)

In [75]:
max(i[0] for i in test_y)

0.0005482964397879859